# Index

1. [Imports](#Imports)
2. [Dataset Importation](#Dataset-Importation)
3. [Data Exploration](#Data-Exploration)
4. [Data Cleaning](#Data-Cleaning)
5. [Sampling patients to reduce data size](#Sampling-patients-to-reduce-data-size)
   - [Checking distribution from original to sampled](#Checking-distribution-from-original-to-sampled)
   - [Final Patients](#Final-Patients)
6. [Preprocessing (patient data)](#Preprocessing-patient-data)
   - [Demographic Data](#Demographic-Data)
7. [Creating the Data Set (Images)](#Creating-the-Data-Set-Images)
     - [Preprocess images](#Preprocess-images)
     - [Split the data](#Split-the-data)
7. [Data Augmentation for Training and Validation](#Data-Augmentation-for-Training-and-Validation)
   - [Visualizing the final augmented images and the bboxes](#Visualizing-the-final-augmented-images-and-the-bboxes)
   - [Dataset](#Dataset)

## Imports

In [1]:
# OS and File Management
import os
import shutil
import random
from glob import glob
import pickle
import xml.etree.ElementTree as ET
import re
import os
from collections import defaultdict
import re

# Scientific Computing and Data Processing
import numpy as np
import pandas as pd

# Image Processing and Augmentation
import cv2
import pydicom
import albumentations as A
from albumentations import (
    Compose, HorizontalFlip, RandomBrightnessContrast, Affine,
    GaussianBlur, CLAHE, RandomGamma, GaussNoise
)

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Machine Learning Utilities
from sklearn.model_selection import train_test_split
from collections import defaultdict

# Custom Utilities
from VisualizationTools.get_data_from_XML import XML_preprocessor, get_category
from VisualizationTools.get_gt import get_gt
from VisualizationTools.getUID import getUID_path
from VisualizationTools.utils import loadFileInformation
from functions_DataPreprocessing import *

import os
import shutil
from collections import defaultdict
from glob import glob
import os
import cv2
import random
import numpy as np
from glob import glob
from tqdm import tqdm
from collections import defaultdict
from albumentations import (
    Compose, HorizontalFlip, RandomBrightnessContrast, Affine,
    GaussianBlur, CLAHE, RandomGamma
)

/Users/catarinasilva/anaconda3/lib/python3.11/site-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.5' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


### Dataset Importation

In [2]:
df = pd.read_excel('/Users/catarinasilva/Desktop/Master Thesis/lung_cancer/statistics_clinical.xlsx')

## Preprocessing (patient data)

In [3]:
patient_ids = ['E0001', 'E0002', 'E0003', 'E0004', 'E0005', 'B0001', 'B0002', 'B0003', 'B0004', 'B0005', 'B0006', 'B0007', 'B0008', 'B0009', 'B0011', 'B0013', 'B0014', 'B0015', 'B0016', 'B0017', 'B0018', 'B0019', 'B0020', 'B0021', 'B0022', 'B0023', 'B0024', 'B0025', 'B0026', 'B0027', 'B0028', 'B0031', 'B0033', 'B0034', 'B0036', 'B0037', 'B0038', 'B0040', 'B0041', 'B0042', 'B0043', 'B0044','A0001', 'A0003', 'A0005', 'A0006', 'A0008', 'A0010', 'A0012', 'A0013', 'A0014', 'A0016', 'A0017', 'A0019', 'A0021', 'A0022', 'A0024', 'A0026', 'A0027', 'A0029', 'A0030', 'A0032', 'A0036', 'A0037', 'A0039', 'A0042', 'A0044', 'A0045', 'A0047', 'A0048', 'A0051', 'A0053', 'A0057', 'A0059', 'A0061', 'A0066', 'A0068', 'A0073', 'A0075', 'A0077', 'A0086', 'A0089', 'A0091', 'A0092', 'A0095', 'A0097', 'A0103', 'A0105', 'A0106', 'A0108', 'A0109', 'A0112', 'A0118', 'A0120', 'A0121', 'A0125', 'A0128', 'A0132', 'A0135', 'A0137', 'A0140', 'A0147', 'A0149', 'A0157', 'A0160', 'A0162', 'A0163', 'A0168', 'A0169', 'A0170', 'A0171', 'A0179', 'A0182', 'A0183', 'A0184', 'A0185', 'A0187', 'A0189', 'A0192', 'G0001', 'G0002', 'G0003', 'G0004', 'G0005', 'G0006', 'G0007', 'G0008', 'G0010', 'G0011', 'G0012', 'G0013', 'G0014', 'G0015', 'G0017', 'G0018', 'G0019', 'G0024', 'G0025', 'G0026', 'G0028', 'G0029', 'G0030', 'G0031', 'G0032', 'G0033', 'G0036', 'G0039', 'G0040', 'G0041', 'G0042', 'G0043', 'G0044', 'G0046', 'G0049', 'G0050', 'G0056', 'G0062']

In [4]:
df_final = df[df['NewPatientID'].isin(patient_ids)]

In [5]:
df_final.shape

(157, 10)

In [6]:
df_final.describe()

,No.,Age,weight (kg),N-Stage,Smoking History
count,157.000000,157.000000,155.000000,157.000000,157.000000
mean,192.006369,60.076433,65.046452,1.280255,0.477707
std,114.528996,10.182653,11.675917,1.270142,0.501101
min,1.000000,28.000000,35.500000,0.000000,0.000000
25%,85.000000,54.000000,57.000000,0.000000,0.000000
50%,253.000000,61.000000,65.000000,1.000000,0.000000
75%,293.000000,67.000000,71.000000,3.000000,1.000000
max,355.000000,83.000000,98.000000,3.000000,1.000000


In [7]:
df_final.describe(include='object')

,NewPatientID,Sex,T-Stage,Ｍ-Stage,Histopathological grading
count,157,157,157,157,82
unique,157,2,10,5,7
top,A0001,M,1c,0,G3
freq,1,91,34,88,54


In [8]:
total_observations = len(df_final)
nan_info_list = []

for column in df_final.columns:
    nan_count = df_final[column].isna().sum()
    nan_percentage = (nan_count / total_observations) * 100
    nan_info_list.append({'Column Name': column,
                          'NaN Count': nan_count,
                          'Percentage of NaNs': f"{nan_percentage:.2f}%",
                          'Total Observations': total_observations})

nan_info = pd.DataFrame(nan_info_list)

display(nan_info)

,Column Name,NaN Count,Percentage of NaNs,Total Observations
0,No.,0,0.00%,157
1,NewPatientID,0,0.00%,157
2,Sex,0,0.00%,157
3,Age,0,0.00%,157
4,weight (kg),2,1.27%,157
5,T-Stage,0,0.00%,157
6,N-Stage,0,0.00%,157
7,Ｍ-Stage,0,0.00%,157
8,Histopathological grading,75,47.77%,157
9,Smoking History,0,0.00%,157


In [9]:
df_final['T-Stage'].value_counts()

T-Stage
1c    34
3     31
2     29
2a    21
4     15
1b    14
1a     6
2b     5
is     1
1      1
Name: count, dtype: int64

In [10]:
df_final['N-Stage'].value_counts()

N-Stage
0    60
3    50
1    43
2     4
Name: count, dtype: int64

In [11]:
df_final['Ｍ-Stage'].value_counts()

Ｍ-Stage
0     88
1a    21
1b    19
1     19
1c    10
Name: count, dtype: int64

In [12]:
df_final = df_final.rename(columns={'NewPatientID': 'PatientID', 'Age': 'age', 'Sex': 'gender'})

In [13]:
df_final.columns

Index(['No.', 'PatientID', 'gender', 'age', 'weight (kg)', 'T-Stage',
       'N-Stage', 'Ｍ-Stage', 'Histopathological grading', 'Smoking History'],
      dtype='object')

## NSCLC Dataset

### Demografic Data

In [14]:
df = pd.read_csv('/Users/catarinasilva/Desktop/Master Thesis/NSCLC data/NSCLC-Radiomics-Lung1.clinical-version3-Oct-2019.csv')

In [15]:
# List of selected PatientIDs for class G
selected_patients_g = [
    'LUNG1-007', 'LUNG1-010', 'LUNG1-014', 'LUNG1-027', 'LUNG1-050', 'LUNG1-061', 'LUNG1-063', 'LUNG1-064', 
    'LUNG1-078', 'LUNG1-082', 'LUNG1-086', 'LUNG1-098', 'LUNG1-122', 'LUNG1-135', 'LUNG1-145', 'LUNG1-146', 
    'LUNG1-150', 'LUNG1-167', 'LUNG1-202', 'LUNG1-206', 'LUNG1-222', 'LUNG1-253', 'LUNG1-264', 'LUNG1-266', 
    'LUNG1-295', 'LUNG1-296', 'LUNG1-302', 'LUNG1-321', 'LUNG1-325', 'LUNG1-326', 'LUNG1-329', 'LUNG1-334', 
    'LUNG1-339', 'LUNG1-346', 'LUNG1-347', 'LUNG1-355', 'LUNG1-356', 'LUNG1-358', 'LUNG1-359', 'LUNG1-362', 
    'LUNG1-365', 'LUNG1-366', 'LUNG1-369', 'LUNG1-371', 'LUNG1-374', 'LUNG1-378', 'LUNG1-379', 'LUNG1-380', 
    'LUNG1-383', 'LUNG1-386', 'LUNG1-389', 'LUNG1-391', 'LUNG1-399', 'LUNG1-404', 'LUNG1-413', 'LUNG1-420'
]

# Filter the dataset to include:
filtered_df = df[(df['PatientID'].isin(selected_patients_g)) | (df['Histology'] == 'large cell')]

# Display the filtered DataFrame
display(filtered_df)

,PatientID,age,clinical.T.Stage,Clinical.N.Stage,Clinical.M.Stage,Overall.Stage,Histology,gender,Survival.time,deadstatus.event
0,LUNG1-001,78.7515,2.0,3,0,IIIb,large cell,male,2165,1
2,LUNG1-003,68.1807,2.0,3,0,IIIb,large cell,male,256,1
6,LUNG1-007,81.5288,2.0,2,0,IIIa,squamous cell carcinoma,male,137,1
9,LUNG1-010,71.0554,4.0,3,0,IIIb,squamous cell carcinoma,female,2119,0
13,LUNG1-014,66.7707,4.0,0,0,IIIb,squamous cell carcinoma,male,1247,1
...,...,...,...,...,...,...,...,...,...,...
390,LUNG1-391,60.0822,2.0,0,0,I,squamous cell carcinoma,female,256,1
398,LUNG1-399,77.7973,1.0,2,0,IIIa,squamous cell carcinoma,female,2835,0
403,LUNG1-404,74.2356,3.0,2,0,IIIa,squamous cell carcinoma,male,280,1
412,LUNG1-413,60.5315,4.0,2,0,IIIb,squamous cell carcinoma,female,246,1


In [16]:
filtered_df.head()

,PatientID,age,clinical.T.Stage,Clinical.N.Stage,Clinical.M.Stage,Overall.Stage,Histology,gender,Survival.time,deadstatus.event
0,LUNG1-001,78.7515,2.0,3,0,IIIb,large cell,male,2165,1
2,LUNG1-003,68.1807,2.0,3,0,IIIb,large cell,male,256,1
6,LUNG1-007,81.5288,2.0,2,0,IIIa,squamous cell carcinoma,male,137,1
9,LUNG1-010,71.0554,4.0,3,0,IIIb,squamous cell carcinoma,female,2119,0
13,LUNG1-014,66.7707,4.0,0,0,IIIb,squamous cell carcinoma,male,1247,1


In [17]:
filtered_df.columns

Index(['PatientID', 'age', 'clinical.T.Stage', 'Clinical.N.Stage',
       'Clinical.M.Stage', 'Overall.Stage', 'Histology', 'gender',
       'Survival.time', 'deadstatus.event'],
      dtype='object')

In [18]:
filtered_df.shape

(170, 10)

In [19]:
filtered_df['Histology'].value_counts()

Histology
large cell                 114
squamous cell carcinoma     56
Name: count, dtype: int64

In [20]:
total_observations = len(filtered_df)
nan_info_list = []

for column in filtered_df.columns:
    nan_count = filtered_df[column].isna().sum()
    nan_percentage = (nan_count / total_observations) * 100
    nan_info_list.append({'Column Name': column,
                          'NaN Count': nan_count,
                          'Percentage of NaNs': f"{nan_percentage:.2f}%",
                          'Total Observations': total_observations})

nan_info = pd.DataFrame(nan_info_list)

display(nan_info)

,Column Name,NaN Count,Percentage of NaNs,Total Observations
0,PatientID,0,0.00%,170
1,age,5,2.94%,170
2,clinical.T.Stage,0,0.00%,170
3,Clinical.N.Stage,0,0.00%,170
4,Clinical.M.Stage,0,0.00%,170
5,Overall.Stage,1,0.59%,170
6,Histology,0,0.00%,170
7,gender,0,0.00%,170
8,Survival.time,0,0.00%,170
9,deadstatus.event,0,0.00%,170


In [21]:
filtered_df.isna().sum()

PatientID           0
age                 5
clinical.T.Stage    0
Clinical.N.Stage    0
Clinical.M.Stage    0
Overall.Stage       1
Histology           0
gender              0
Survival.time       0
deadstatus.event    0
dtype: int64

In [22]:
rows_with_nan = filtered_df[filtered_df[['Overall.Stage', 'age']].isna().any(axis=1)]

display(rows_with_nan)

,PatientID,age,clinical.T.Stage,Clinical.N.Stage,Clinical.M.Stage,Overall.Stage,Histology,gender,Survival.time,deadstatus.event
271,LUNG1-272,60.1396,5.0,2,0,NaN,large cell,male,288,1
274,LUNG1-275,NaN,2.0,3,0,IIIb,large cell,male,173,1
302,LUNG1-303,NaN,2.0,0,0,I,large cell,male,24,1
307,LUNG1-308,NaN,2.0,1,0,II,large cell,female,213,1
338,LUNG1-339,NaN,4.0,2,0,IIIb,squamous cell carcinoma,male,120,1
353,LUNG1-354,NaN,1.0,2,0,IIIa,large cell,female,617,1


In [23]:
filtered_df.describe()

,age,clinical.T.Stage,Clinical.N.Stage,Clinical.M.Stage,Survival.time,deadstatus.event
count,165.000000,170.000000,170.000000,170.000000,170.000000,170.000000
mean,67.681981,2.482353,1.594118,0.035294,1071.370588,0.870588
std,10.231576,1.152427,1.194153,0.324432,1041.800460,0.336647
min,33.684900,1.000000,0.000000,0.000000,10.000000,0.000000
25%,60.695900,2.000000,0.000000,0.000000,265.250000,1.000000
50%,68.106800,2.000000,2.000000,0.000000,637.000000,1.000000
75%,74.929500,4.000000,2.750000,0.000000,1595.750000,1.000000
max,91.704300,5.000000,4.000000,3.000000,4328.000000,1.000000


In [24]:
filtered_df.describe(include='object')

,PatientID,Overall.Stage,Histology,gender
count,170,169,170,170
unique,170,4,2,2
top,LUNG1-001,IIIb,large cell,male
freq,1,76,114,106


In [25]:
filtered_df['Histology'].value_counts()

Histology
large cell                 114
squamous cell carcinoma     56
Name: count, dtype: int64

In [26]:
filtered_df = filtered_df.rename(columns={'clinical.T.Stage': 'T-Stage', 'Clinical.N.Stage': 'N-Stage', 'Clinical.M.Stage': 'Ｍ-Stage'})

In [27]:
filtered_df.columns

Index(['PatientID', 'age', 'T-Stage', 'N-Stage', 'Ｍ-Stage', 'Overall.Stage',
       'Histology', 'gender', 'Survival.time', 'deadstatus.event'],
      dtype='object')

In [28]:
filtered_df['T-Stage'].value_counts()

T-Stage
2.0    64
4.0    50
1.0    38
3.0    17
5.0     1
Name: count, dtype: int64

In [29]:
filtered_df['N-Stage'].value_counts()

N-Stage
2    66
0    53
3    41
1     8
4     2
Name: count, dtype: int64

In [30]:
filtered_df['Ｍ-Stage'].value_counts()

Ｍ-Stage
0    168
3      2
Name: count, dtype: int64

In [31]:
filtered_df['Overall.Stage'].value_counts()

Overall.Stage
IIIb    76
IIIa    55
I       27
II      11
Name: count, dtype: int64

### Joining bothe DFs

In [32]:
# Concatenar os dois DataFrames
combined_df = pd.concat([df_final, filtered_df], ignore_index=True)

# Preencher NaNs com 'Not specified'
df_concat = combined_df.fillna('Not specified')

In [33]:
df_concat.head()

,No.,PatientID,gender,age,weight (kg),T-Stage,N-Stage,Ｍ-Stage,Histopathological grading,Smoking History,Overall.Stage,Histology,Survival.time,deadstatus.event
0,1.0,A0001,M,58.0,65.0,2b,3,1b,G3,1.0,Not specified,Not specified,Not specified,Not specified
1,3.0,A0003,M,60.0,62.0,1c,1,0,G3,0.0,Not specified,Not specified,Not specified,Not specified
2,5.0,A0005,M,70.0,75.0,1b,0,0,G2,1.0,Not specified,Not specified,Not specified,Not specified
3,6.0,A0006,F,36.0,57.0,1b,2,0,G3,0.0,Not specified,Not specified,Not specified,Not specified
4,8.0,A0008,F,66.0,60.5,2,0,1,Not specified,0.0,Not specified,Not specified,Not specified,Not specified


In [34]:
df_concat['T-Stage'].value_counts()

T-Stage
2      93
4      65
3      48
1      39
1c     34
2a     21
1b     14
1a      6
2b      5
is      1
5.0     1
Name: count, dtype: int64

In [35]:
df_concat[df_concat['T-Stage'] == 5.0]

,No.,PatientID,gender,age,weight (kg),T-Stage,N-Stage,Ｍ-Stage,Histopathological grading,Smoking History,Overall.Stage,Histology,Survival.time,deadstatus.event
267,Not specified,LUNG1-272,male,60.1396,Not specified,5.0,2,0,Not specified,Not specified,Not specified,large cell,288.0,1.0


In [36]:
df_concat[df_concat['T-Stage'] == 'is']

,No.,PatientID,gender,age,weight (kg),T-Stage,N-Stage,Ｍ-Stage,Histopathological grading,Smoking History,Overall.Stage,Histology,Survival.time,deadstatus.event
12,21.0,A0021,F,42.0,58.0,is,0,0,Not specified,0.0,Not specified,Not specified,Not specified,Not specified


In [37]:
df_concat = df_concat[df_concat['T-Stage'] != 5.0]
df_concat = df_concat[df_concat['T-Stage'] != 'is']

In [38]:
df_concat['T-Stage'].value_counts()

T-Stage
2     93
4     65
3     48
1     39
1c    34
2a    21
1b    14
1a     6
2b     5
Name: count, dtype: int64

In [39]:
df_concat['N-Stage'].value_counts()

N-Stage
0    112
3     91
2     69
1     51
4      2
Name: count, dtype: int64

In [40]:
df_concat[df_concat['N-Stage'] == 4]

,No.,PatientID,gender,age,weight (kg),T-Stage,N-Stage,Ｍ-Stage,Histopathological grading,Smoking History,Overall.Stage,Histology,Survival.time,deadstatus.event
212,Not specified,LUNG1-167,male,74.9295,Not specified,4.0,4,0,Not specified,Not specified,IIIb,squamous cell carcinoma,1357.0,1.0
247,Not specified,LUNG1-232,male,73.2868,Not specified,4.0,4,0,Not specified,Not specified,IIIb,large cell,2521.0,1.0


In [41]:
df_concat.loc[df_concat['N-Stage'] == 4, 'N-Stage'] = 2

In [42]:
df_concat['N-Stage'].value_counts()

N-Stage
0    112
3     91
2     71
1     51
Name: count, dtype: int64

In [43]:
df_concat['Ｍ-Stage'].value_counts()

Ｍ-Stage
0     254
1a     21
1b     19
1      19
1c     10
3       2
Name: count, dtype: int64

In [44]:
df_concat[df_concat['Ｍ-Stage'] == 3]

,No.,PatientID,gender,age,weight (kg),T-Stage,N-Stage,Ｍ-Stage,Histopathological grading,Smoking History,Overall.Stage,Histology,Survival.time,deadstatus.event
259,Not specified,LUNG1-256,male,53.0842,Not specified,4.0,2,3,Not specified,Not specified,IIIb,large cell,291.0,1.0
265,Not specified,LUNG1-269,male,73.0595,Not specified,3.0,3,3,Not specified,Not specified,IIIa,large cell,193.0,1.0


In [45]:
df_concat.loc[df_concat['Ｍ-Stage'] == 3, 'Ｍ-Stage'] = 0

In [46]:
df_concat['Ｍ-Stage'].value_counts()

Ｍ-Stage
0     256
1a     21
1b     19
1      19
1c     10
Name: count, dtype: int64

In [47]:
df_concat = df_concat.rename(columns={'Ｍ-Stage': 'M-Stage'})

In [48]:
df_concat.head(2)

,No.,PatientID,gender,age,weight (kg),T-Stage,N-Stage,M-Stage,Histopathological grading,Smoking History,Overall.Stage,Histology,Survival.time,deadstatus.event
0,1.0,A0001,M,58.0,65.0,2b,3,1b,G3,1.0,Not specified,Not specified,Not specified,Not specified
1,3.0,A0003,M,60.0,62.0,1c,1,0,G3,0.0,Not specified,Not specified,Not specified,Not specified


In [49]:
def fill_histology(row):
    if row['Histology'] != 'Not specified':
        return row['Histology']
    
    pid = row['PatientID']
    if pid.startswith('A'):
        return 'Adenocarcinoma'
    elif pid.startswith('B'):
        return 'small cell'
    elif pid.startswith('G'):
        return 'squamous cell carcinoma'
    elif pid.startswith('E'):
        return 'large cell'
    else:
        return 'Unknown'  # caso de segurança

df_concat['Histology'] = df_concat.apply(fill_histology, axis=1)

In [50]:
df_concat['Histology'].value_counts()

Histology
large cell                 118
squamous cell carcinoma     94
Adenocarcinoma              76
small cell                  37
Name: count, dtype: int64

In [51]:
df_concat.loc[df_concat['Histology'] =='large cell', 'Histology'] = 0
df_concat.loc[df_concat['Histology'] == 'squamous cell carcinoma', 'Histology'] = 1
df_concat.loc[df_concat['Histology'] =='Adenocarcinoma', 'Histology'] = 2
df_concat.loc[df_concat['Histology'] == 'small cell', 'Histology'] = 3

In [52]:
df_concat['Histology'].value_counts()

Histology
0    118
1     94
2     76
3     37
Name: count, dtype: int64

In [53]:
df_concat.isna().sum()

No.                          0
PatientID                    0
gender                       0
age                          0
weight (kg)                  0
T-Stage                      0
N-Stage                      0
M-Stage                      0
Histopathological grading    0
Smoking History              0
Overall.Stage                0
Histology                    0
Survival.time                0
deadstatus.event             0
dtype: int64

In [54]:
df_concat['weight (kg)'].value_counts()

weight (kg)
Not specified    171
65.0              18
75.0              10
60.0              10
70.0               9
57.0               7
55.0               7
67.0               6
58.0               5
68.0               5
45.0               4
52.0               4
90.0               4
63.0               4
66.0               4
53.0               4
50.0               4
71.0               3
69.0               3
79.0               2
88.0               2
51.0               2
80.0               2
62.5               2
78.0               2
46.0               2
35.5               1
98.0               1
73.0               1
40.0               1
48.0               1
76.5               1
84.0               1
85.0               1
56.0               1
72.5               1
47.0               1
72.0               1
51.8               1
84.5               1
76.4               1
44.0               1
82.0               1
66.5               1
60.5               1
93.0               1
67.5               1
7

In [55]:
# Convert the column to float (it will convert all non-numeric values to NaN automatically)
df_concat['weight (kg)'] = pd.to_numeric(df_concat['weight (kg)'], errors='coerce')

# Compute the mean of the numeric (non-NaN) values
mean_weight = df_concat['weight (kg)'].mean()

# Fill NaN values (including those from 'Not specified') with the mean
df_concat['weight (kg)'] = df_concat['weight (kg)'].fillna(mean_weight)

# Arredondar para as unidades e converter para int
df_concat['weight (kg)'] = df_concat['weight (kg)'].round(0).astype(int)

In [56]:
df_concat['weight (kg)'].value_counts()

weight (kg)
65    189
60     11
75     10
70      9
57      7
55      7
68      6
67      6
52      5
66      5
58      5
45      4
63      4
90      4
50      4
53      4
71      3
69      3
76      3
62      3
84      2
80      2
51      2
72      2
79      2
46      2
78      2
88      2
56      1
36      1
73      1
48      1
40      1
85      1
47      1
82      1
98      1
44      1
93      1
74      1
87      1
86      1
81      1
49      1
64      1
Name: count, dtype: int64

In [57]:
df_concat['age'].value_counts()

age
60.0000    12
64.0000     9
67.0000     9
65.0000     8
58.0000     7
           ..
69.8946     1
85.4949     1
67.4771     1
64.7255     1
73.3808     1
Name: count, Length: 208, dtype: int64

In [58]:
# Convert the column to float (it will convert all non-numeric values to NaN automatically)
df_concat['age'] = pd.to_numeric(df_concat['age'], errors='coerce')

# Compute the mean of the numeric (non-NaN) values
mean_weight = df_concat['age'].mean()

# Fill NaN values (including those from 'Not specified') with the mean
df_concat['age'] = df_concat['age'].fillna(mean_weight)

# Arredondar para as unidades e converter para int
df_concat['age'] = df_concat['age'].round(0).astype(int)

In [59]:
df_concat['gender'].value_counts()

gender
male      105
M          91
F          65
female     64
Name: count, dtype: int64

In [60]:
df_concat.loc[df_concat['gender'] =='M', 'gender'] = 'male'
df_concat.loc[df_concat['gender'] == 'F', 'gender'] = 'female'

In [61]:
df_concat['gender'].value_counts()

gender
male      196
female    129
Name: count, dtype: int64

In [62]:
df_concat.loc[df_concat['gender'] =='male', 'gender'] = 0
df_concat.loc[df_concat['gender'] == 'female', 'gender'] = 1

In [63]:
df_concat['gender'].value_counts()

gender
0    196
1    129
Name: count, dtype: int64

In [64]:
df_concat['Smoking History'].value_counts()

Smoking History
Not specified    169
0.0               81
1.0               75
Name: count, dtype: int64

In [65]:
df_concat.loc[df_concat['Smoking History'] == 'Not specified', 'Smoking History'] = 2

In [66]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_concat[['age', 'weight (kg)']] = scaler.fit_transform(df_concat[['age', 'weight (kg)']])

#### Encoding target

In [67]:
df_concat['T-Stage'].value_counts()

T-Stage
2     93
4     65
3     48
1     39
1c    34
2a    21
1b    14
1a     6
2b     5
Name: count, dtype: int64

In [68]:
t_stage_mapping = {
    '1': 0, '1.0': 0,
    '1a': 0,
    '1b': 0,
    '1c': 0,
    '2': 1, '2.0': 1,
    '2a': 1,
    '2b': 1,
    '3': 2, '3.0': 2,
    '4': 3, '4.0': 3
}

# Apply mapping
df_concat['T-Stage'] = df_concat['T-Stage'].astype(str).map(t_stage_mapping)

In [69]:
df_concat['T-Stage'].value_counts()

T-Stage
1    119
0     93
3     65
2     48
Name: count, dtype: int64

In [70]:
df_concat['M-Stage'].value_counts()

M-Stage
0     256
1a     21
1b     19
1      19
1c     10
Name: count, dtype: int64

In [71]:
# Convert to string for consistent comparison
df_concat['M-Stage'] = df_concat['M-Stage'].astype(str).apply(lambda x: 0 if x == '0' or x == '0.0' else 1)

In [72]:
df_concat['M-Stage'].value_counts()

M-Stage
0    256
1     69
Name: count, dtype: int64

In [73]:
df_concat['N-Stage'].value_counts()

N-Stage
0    112
3     91
2     71
1     51
Name: count, dtype: int64

In [74]:
df_concat.head()

,No.,PatientID,gender,age,weight (kg),T-Stage,N-Stage,M-Stage,Histopathological grading,Smoking History,Overall.Stage,Histology,Survival.time,deadstatus.event
0,1.0,A0001,0,-0.564874,-0.004223,1,3,1,G3,1.0,Not specified,2,Not specified,Not specified
1,3.0,A0003,0,-0.378589,-0.378534,0,1,0,G3,0.0,Not specified,2,Not specified,Not specified
2,5.0,A0005,0,0.552837,1.243481,0,0,0,G2,1.0,Not specified,2,Not specified,Not specified
3,6.0,A0006,1,-2.614011,-1.002386,0,2,0,G3,0.0,Not specified,2,Not specified,Not specified
4,8.0,A0008,1,0.180267,-0.628075,1,0,1,Not specified,0.0,Not specified,2,Not specified,Not specified


In [75]:
df_concat.columns

Index(['No.', 'PatientID', 'gender', 'age', 'weight (kg)', 'T-Stage',
       'N-Stage', 'M-Stage', 'Histopathological grading', 'Smoking History',
       'Overall.Stage', 'Histology', 'Survival.time', 'deadstatus.event'],
      dtype='object')

In [76]:
df_concat.to_csv('TMN_LUNGPETDX_NSCLC_finaldataset.csv')

### Spliting

In [80]:
def count_images_per_target_stage(image_dir, df, stage_col):
    """
    Conta o número de imagens por classe de um determinado estágio (T/N/M).

    Args:
        image_dir (str): Caminho para a pasta com imagens (.jpg).
        df (pd.DataFrame): DataFrame com colunas ['PatientID', 'T-Stage', 'N-Stage', 'M-Stage'].
        stage_col (str): Nome da coluna alvo ('T-Stage', 'N-Stage' ou 'M-Stage').

    Returns:
        dict: Contagem de imagens por classe do estágio.
    """
    df = df.copy()
    df[stage_col] = df[stage_col].astype(str)

    stage_counts = defaultdict(int)

    image_paths = glob(os.path.join(image_dir, '*.jpg'))

    for img_path in image_paths:
        patient_id = extract_patient_id(img_path)
        if patient_id and patient_id in df['PatientID'].values:
            stage_value = df.loc[df['PatientID'] == patient_id, stage_col].values[0]
            stage_counts[stage_value] += 1

    return dict(stage_counts)


from collections import defaultdict
import os
from glob import glob

def count_images_per_TMN_combination(image_dir, df):
    """
    Conta o número de imagens por combinação de T-Stage, N-Stage e M-Stage.

    Args:
        image_dir (str): Caminho para a pasta com imagens (.jpg).
        df (pd.DataFrame): DataFrame com colunas ['PatientID', 'T-Stage', 'N-Stage', 'M-Stage'].

    Returns:
        dict: Contagem de imagens por combinação (T, N, M).
    """
    df = df.copy()
    df = df[df['PatientID'].notna()]
    df[['T-Stage', 'N-Stage', 'M-Stage']] = df[['T-Stage', 'N-Stage', 'M-Stage']].astype(str)

    combo_counts = defaultdict(int)
    image_paths = glob(os.path.join(image_dir, '*.jpg'))

    for img_path in image_paths:
        patient_id = extract_patient_id(img_path)
        if patient_id and patient_id in df['PatientID'].values:
            row = df[df['PatientID'] == patient_id].iloc[0]
            combo = (row['T-Stage'], row['N-Stage'], row['M-Stage'])
            combo_counts[combo] += 1

    return dict(combo_counts)


def extract_patient_id(filename):
    name = os.path.splitext(os.path.basename(filename))[0]
    match = re.match(r'^([ABEG]\d{3,4})_', name)
    if match:
        return match.group(1)
    match = re.match(r'^[EG]_(LUNG\d+-\d+)_', name)
    if match:
        return match.group(1)
    return None

In [82]:
image_dir='/Users/catarinasilva/Desktop/Master Thesis/lung_cancer/split patient E/train/images'

# Contar imagens por T-stage
t_counts = count_images_per_target_stage(image_dir, df_concat, stage_col='T-Stage')
print("T-Stage image counts:", t_counts)

# Contar imagens por N-stage
n_counts = count_images_per_target_stage(image_dir, df_concat, stage_col='N-Stage')
print("N-Stage image counts:", n_counts)

# Contar imagens por M-stage
m_counts = count_images_per_target_stage(image_dir, df_concat, stage_col='M-Stage')
print("M-Stage image counts:", m_counts)

T-Stage image counts: {'1': 3562, '3': 1429, '0': 1802, '2': 2441}
N-Stage image counts: {'0': 3543, '3': 1967, '1': 2684, '2': 1040}
M-Stage image counts: {'1': 2866, '0': 6368}


In [83]:
image_dir='/Users/catarinasilva/Desktop/Master Thesis/lung_cancer/split patient E/val/images'

# Contar imagens por T-stage
t_counts = count_images_per_target_stage(image_dir, df_concat, stage_col='T-Stage')
print("T-Stage image counts:", t_counts)

# Contar imagens por N-stage
n_counts = count_images_per_target_stage(image_dir, df_concat, stage_col='N-Stage')
print("N-Stage image counts:", n_counts)

# Contar imagens por M-stage
m_counts = count_images_per_target_stage(image_dir, df_concat, stage_col='M-Stage')
print("M-Stage image counts:", m_counts)

T-Stage image counts: {'1': 683, '2': 542, '0': 362, '3': 250}
N-Stage image counts: {'3': 268, '0': 797, '1': 604, '2': 168}
M-Stage image counts: {'0': 1373, '1': 464}


In [84]:
image_dir='/Users/catarinasilva/Desktop/Master Thesis/lung_cancer/split patient E/test/images'

# Contar imagens por T-stage
t_counts = count_images_per_target_stage(image_dir, df_concat, stage_col='T-Stage')
print("T-Stage image counts:", t_counts)

# Contar imagens por N-stage
n_counts = count_images_per_target_stage(image_dir, df_concat, stage_col='N-Stage')
print("N-Stage image counts:", n_counts)

# Contar imagens por M-stage
m_counts = count_images_per_target_stage(image_dir, df_concat, stage_col='M-Stage')
print("M-Stage image counts:", m_counts)

T-Stage image counts: {'3': 167, '0': 752, '1': 642, '2': 126}
N-Stage image counts: {'3': 215, '0': 577, '1': 726, '2': 169}
M-Stage image counts: {'1': 323, '0': 1364}


### Augmentation

In [149]:
def remove_majority_combination_images(df, image_dir, t_major='0', n_major='3', m_major='0', target_to_remove=100):
    """
    Remove imagens da combinação T0-N3-M0 após augmentação, priorizando imagens aumentadas e pacientes com mais imagens.

    Args:
        df (pd.DataFrame): DataFrame com info dos pacientes.
        image_dir (str): Diretório onde estão as imagens (e.g., .../images/train).
        t_major (str): Classe majoritária de T-Stage.
        n_major (str): Classe majoritária de N-Stage.
        m_major (str): Classe majoritária de M-Stage.
        target_to_remove (int): Nº total de imagens a remover.
    """
    print(f"\n🚨 Initiating removal for majority class combo: T={t_major}, N={n_major}, M={m_major}")
    removed_count = 0

    # Mapear imagens por paciente
    image_paths = glob(os.path.join(image_dir, "*.jpg"))
    patient_to_images = defaultdict(list)
    for img_path in image_paths:
        fname = os.path.basename(img_path)
        pid = extract_patient_id(fname)
        if pid:
            patient_to_images[pid].append(img_path)

    # Identificar pacientes da classe majoritária combinada
    majority_patients = df[
        (df['T-Stage'].astype(str) == t_major) &
        (df['N-Stage'].astype(str) == n_major) &
        (df['M-Stage'].astype(str) == m_major)
    ]['PatientID'].tolist()

    print(f"  🔍 Found {len(majority_patients)} patients with majority combo")

    # Ordenar por número de imagens (priorizar quem tem mais)
    sorted_patients = sorted(
        majority_patients,
        key=lambda pid: len(patient_to_images.get(pid, [])),
        reverse=True
    )

    for pid in sorted_patients:
        imgs = patient_to_images.get(pid, [])
        if not imgs:
            continue

        # Priorizar imagens aumentadas
        aug_imgs = [img for img in imgs if '_aug_' in os.path.basename(img)]
        orig_imgs = [img for img in imgs if '_aug_' not in os.path.basename(img)]

        for img_path in aug_imgs + orig_imgs:
            if removed_count >= target_to_remove:
                break
            os.remove(img_path)
            removed_count += 1

        if removed_count >= target_to_remove:
            break

    print(f"✅ Removed {removed_count} images from majority class combination.")

import re

def extract_patient_id(filename):
    name = os.path.splitext(os.path.basename(filename))[0]
    match = re.match(r'^([ABEG]\d{3,4})_', name)
    if match:
        return match.group(1)
    match = re.match(r'^[EG]_(LUNG\d+-\d+)_', name)
    if match:
        return match.group(1)
    return None

def augment_specific_class_combination(
    input_dir,
    df,
    target_class=('1', '3', '0'),
    num_aug_images=100
):
    augmentation = Compose([
        HorizontalFlip(p=0.3),
        RandomBrightnessContrast(p=0.4),
        Affine(translate_percent=(0.05, 0.1), scale=(0.85, 1.2), rotate=(-15, 15), shear=(-5, 5), p=0.7),
        GaussianBlur(blur_limit=(3, 7), p=0.3),
        CLAHE(clip_limit=(1.0, 5.0), p=0.4),
        RandomGamma(p=0.3),
    ])

    df = df[df['PatientID'].notna()].copy()
    for col in ['T-Stage', 'N-Stage', 'M-Stage']:
        df[col] = df[col].astype(str)
    df['class_tuple'] = list(zip(df['T-Stage'], df['N-Stage'], df['M-Stage']))

    df_class = df[df['class_tuple'] == target_class]
    if df_class.empty:
        print(f"❌ No patients found for class {target_class}")
        return

    image_paths = glob(os.path.join(input_dir, '*.jpg'))
    patient_to_imgs = defaultdict(list)

    for path in image_paths:
        pid = extract_patient_id(os.path.basename(path))
        if pid in df_class['PatientID'].values:
            patient_to_imgs[pid].append(path)

    patient_aug_count = defaultdict(int)
    all_augmented = 0

    print(f"\n🚀 Augmenting {num_aug_images} images for class {target_class} in {input_dir}...\n")

    while all_augmented < num_aug_images:
        eligible = sorted(patient_to_imgs.items(), key=lambda kv: len(kv[1]) + patient_aug_count[kv[0]])
        augmented_this_round = False

        for pid, imgs in eligible:
            if all_augmented >= num_aug_images:
                break

            if not imgs:
                continue

            img_path = random.choice(imgs)
            img = cv2.imread(img_path)
            if img is None:
                continue

            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            aug_img = augmentation(image=img)['image']
            aug_img = cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR)

            base = os.path.splitext(os.path.basename(img_path))[0]
            aug_name = f"{base}_aug_{random.randint(1000, 9999)}.jpg"
            aug_path = os.path.join(input_dir, aug_name)

            cv2.imwrite(aug_path, aug_img)
            patient_to_imgs[pid].append(aug_path)
            patient_aug_count[pid] += 1
            all_augmented += 1
            augmented_this_round = True
            print(f"  [+] Augmented for patient {pid} ({all_augmented}/{num_aug_images})")
            break  # apenas 1 por iteração p/ equilíbrio

        if not augmented_this_round:
            print("⚠️ No more augmentable patients/images found.")
            break

    print("\n✅ Augmentation completed.")
    print(f"📊 Final augmented counts per patient (class {target_class}):")
    for pid, count in sorted(patient_aug_count.items(), key=lambda x: -x[1]):
        print(f"  {pid}: {count} augmented")


In [145]:
image_dir='/Users/catarinasilva/Desktop/Master Thesis/lung_cancer/Dataset TMN/train/images'

# Contar imagens por T-stage
t_counts = count_images_per_target_stage(image_dir, df_concat, stage_col='T-Stage')
print("T-Stage image counts:", t_counts)

# Contar imagens por N-stage
n_counts = count_images_per_target_stage(image_dir, df_concat, stage_col='N-Stage')
print("N-Stage image counts:", n_counts)

# Contar imagens por M-stage
m_counts = count_images_per_target_stage(image_dir, df_concat, stage_col='M-Stage')
print("M-Stage image counts:", m_counts)

T-Stage image counts: {'1': 3562, '3': 1429, '0': 1802, '2': 2441}
N-Stage image counts: {'0': 3543, '3': 1967, '1': 2684, '2': 1040}
M-Stage image counts: {'1': 2866, '0': 6368}


In [147]:
image_dir = '/Users/catarinasilva/Desktop/Master Thesis/lung_cancer/Dataset TMN/train/images'  # ou val/test
tmn_image_counts = count_images_per_TMN_combination(image_dir, df_concat)

for combo, count in sorted(tmn_image_counts.items(), key=lambda x: -x[1]):
    print(f"TMN Combo {combo}: {count} images")

TMN Combo ('1', '1', '0'): 930 images
TMN Combo ('1', '0', '0'): 847 images
TMN Combo ('2', '1', '1'): 818 images
TMN Combo ('2', '0', '0'): 759 images
TMN Combo ('1', '3', '0'): 645 images
TMN Combo ('0', '0', '0'): 541 images
TMN Combo ('1', '2', '0'): 512 images
TMN Combo ('3', '0', '0'): 374 images
TMN Combo ('2', '0', '1'): 360 images
TMN Combo ('0', '3', '0'): 359 images
TMN Combo ('1', '0', '1'): 350 images
TMN Combo ('3', '2', '0'): 318 images
TMN Combo ('0', '1', '1'): 315 images
TMN Combo ('3', '1', '0'): 273 images
TMN Combo ('3', '3', '0'): 265 images
TMN Combo ('0', '0', '1'): 256 images
TMN Combo ('1', '3', '1'): 190 images
TMN Combo ('2', '3', '0'): 177 images
TMN Combo ('2', '3', '1'): 164 images
TMN Combo ('0', '3', '1'): 122 images
TMN Combo ('0', '2', '0'): 119 images
TMN Combo ('3', '1', '1'): 98 images
TMN Combo ('1', '1', '1'): 88 images
TMN Combo ('2', '2', '0'): 87 images
TMN Combo ('0', '1', '0'): 86 images
TMN Combo ('2', '1', '0'): 76 images
TMN Combo ('3', '

#### Train

In [170]:
image_dir='/Users/catarinasilva/Desktop/Master Thesis/lung_cancer/Dataset TMN/train/images'

# Contar imagens por T-stage
t_counts = count_images_per_target_stage(image_dir, df_concat, stage_col='T-Stage')
print("T-Stage image counts:", t_counts)

# Contar imagens por N-stage
n_counts = count_images_per_target_stage(image_dir, df_concat, stage_col='N-Stage')
print("N-Stage image counts:", n_counts)

# Contar imagens por M-stage
m_counts = count_images_per_target_stage(image_dir, df_concat, stage_col='M-Stage')
print("M-Stage image counts:", m_counts)

T-Stage image counts: {'1': 2862, '3': 2829, '0': 2802, '2': 2941}
N-Stage image counts: {'0': 2843, '2': 2640, '3': 2967, '1': 2984}
M-Stage image counts: {'1': 5566, '0': 5868}


#### Val

In [254]:
image_dir = '/Users/catarinasilva/Desktop/Master Thesis/lung_cancer/Dataset TMN/val/images'  # ou val/test
tmn_image_counts = count_images_per_TMN_combination(image_dir, df_concat)

for combo, count in sorted(tmn_image_counts.items(), key=lambda x: -x[1]):
    print(f"TMN Combo {combo}: {count} images")

TMN Combo ('2', '1', '1'): 311 images
TMN Combo ('3', '2', '0'): 246 images
TMN Combo ('1', '3', '1'): 212 images
TMN Combo ('2', '0', '1'): 146 images
TMN Combo ('0', '0', '0'): 141 images
TMN Combo ('0', '3', '1'): 116 images
TMN Combo ('3', '1', '1'): 79 images
TMN Combo ('3', '0', '0'): 79 images
TMN Combo ('0', '1', '0'): 78 images
TMN Combo ('1', '2', '0'): 68 images
TMN Combo ('0', '2', '0'): 66 images
TMN Combo ('1', '0', '0'): 54 images
TMN Combo ('2', '2', '0'): 48 images
TMN Combo ('3', '3', '0'): 46 images
TMN Combo ('1', '1', '0'): 36 images
TMN Combo ('1', '3', '0'): 33 images
TMN Combo ('0', '3', '0'): 21 images
TMN Combo ('2', '0', '0'): 17 images
TMN Combo ('2', '3', '0'): 10 images


In [256]:
image_dir='/Users/catarinasilva/Desktop/Master Thesis/lung_cancer/Dataset TMN/val/images'

# Contar imagens por T-stage
t_counts = count_images_per_target_stage(image_dir, df_concat, stage_col='T-Stage')
print("T-Stage image counts:", t_counts)

# Contar imagens por\ N-stage
n_counts = count_images_per_target_stage(image_dir, df_concat, stage_col='N-Stage')
print("N-Stage image counts:", n_counts)

# Contar imagens por M-stage
m_counts = count_images_per_target_stage(image_dir, df_concat, stage_col='M-Stage')
print("M-Stage image counts:", m_counts)

T-Stage image counts: {'1': 403, '2': 532, '3': 450, '0': 422}
N-Stage image counts: {'3': 438, '0': 437, '1': 504, '2': 428}
M-Stage image counts: {'1': 864, '0': 943}
